# Data Collection
This noteboook is responsible for autonomously collecting Twitch follower, concurrent view, and total view data.

## Imports

In [21]:
#SQL connection and queries
import MySQLdb as mdb
#Connection failure exiting
import sys
#Getting current time (after data is scraped) for stream table
from time import gmtime, strftime, time, sleep
#Scraping
import urllib2
from bs4 import BeautifulSoup as bs
#Web automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## File data
Below the database credentials are read in. These are used for logging into the database, creating the necessary tables (if not already created), and storing data.

In [22]:
db_user = ""
db_pass = ""
db_name = ""
db_host = "localhost"
with open("database_credentials.txt") as f:
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_name = f.readline().strip()
#print(db_user, db_pass, db_name)

## Database
### Versioning
Below the database is accessed using the specified username, password, and database name above. Note if the below code errors out, the rest of the database operations in this file will not produce expeted results (most likely will flat out not work).

In [23]:
#Connect
def mdb_get_version():
    """
    Connects to the database above using the obtained credentials.
    """
    con = None
    try:
        con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
        cur = con.cursor()
        cur.execute("SELECT VERSION()")
        data = cur.fetchone()
        print("Database version: {}".format(data))
    except mdb.Error, e:
        print("Error {}: {}".format(e.args[0], e.args[1]))
        sys.exit(1)
    finally:
        if con:
            con.close()
mdb_get_version()

Database version: ('5.7.21-log',)


### Table Creation
The below functions create the necessary tables for data storage if they do not already exist.

In [24]:
def create_tables():
    """
    Creates the necessary tables (if not already created) for Twitch data storage.
    """
    con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    
    with con:
        cur = con.cursor()
        
        #Twitch game/creative/irl categories
        sql = """
        CREATE TABLE IF NOT EXISTS categories (
        id INT PRIMARY KEY AUTO_INCREMENT,
        category VARCHAR(255) NOT NULL UNIQUE
        )
        """
        cur.execute(sql)
        
        #Twitch streamers
        sql = """
        CREATE TABLE IF NOT EXISTS streamers (
        id INT PRIMARY KEY AUTO_INCREMENT,
        streamer VARCHAR(255) NOT NULL UNIQUE
        )
        """
        cur.execute(sql)
        
        #Individual stream
        """
        It's important to note here that any game, like Overwatch, will be streamed by multiple streamers.
        Similarly, a streamer may stream multiple games, or even stream Overwatch at different times.
        The goal of the learning agents will be to see how streaming variables, like stream time, affect
        concurrent viewership, if at all.
        """
        sql = """
        CREATE TABLE IF NOT EXISTS streams (
        id INT PRIMARY KEY AUTO_INCREMENT,
        current_viewers INT NOT NULL DEFAULT -1,
        created_at DATETIME NOT NULL DEFAULT current_timestamp,
        followers INT NOT NULL DEFAULT -1,
        total_views INT NOT NULL DEFAULT -1,
        streamer_id INT NOT NULL DEFAULT -1,
        FOREIGN KEY (streamer_id)
            REFERENCES streamers(id)
            ON DELETE CASCADE,
        category_id INT NOT NULL DEFAULT -1,
        FOREIGN KEY (category_id)
            REFERENCES categories(id)
            ON DELETE CASCADE
        )
        """
        cur.execute(sql)
create_tables()

C:\Users\unknown\AppData\Local\conda\conda\envs\twitch_views\lib\site-packages\ipykernel_launcher.py:17: Warning: Table 'categories' already exists
C:\Users\unknown\AppData\Local\conda\conda\envs\twitch_views\lib\site-packages\ipykernel_launcher.py:26: Warning: Table 'streamers' already exists
C:\Users\unknown\AppData\Local\conda\conda\envs\twitch_views\lib\site-packages\ipykernel_launcher.py:52: Warning: Table 'streams' already exists


## Twitch parameters
Below the variables for scraping on Twitch are setup. Currently this includes the following:
* url strings
* categories
    * these were chosen from the top 10 streaming categories on 2/2/2018 at 2:40pm

### URL strings

In [25]:
#String for vieweing all current streams for a specified category
category_string = "https://www.twitch.tv/directory/game/{}"

#String for navigating to a specific user's stream
stream_string = "https://www.twitch.tv/{}"

### Category strings

In [26]:
#Below is a list of the 10 most popular streaming categories on twitch as of
#2/2/2018 @ 2:40pm
safe_chars = "'"
categories = [
    "fortnite",
    "league of legends",
    "playerunknown's battlegrounds",
    "grand theft auto v",
    "hearthstone",
    "counter-strike: global offensive",
    "dota 2",
    "irl",
    "world of warcraft",
    "overwatch"
]
categories = ["irl"]
category_map = {}

print("Categories:")
for i in range(len(categories)):
    c = categories[i]
    category_map[c] = category_string.format(urllib2.quote(c, safe=safe_chars))
    print("\t{}. {}: {}".format(i, c, category_map[c]))

Categories:
	0. irl: https://www.twitch.tv/directory/game/irl


## Data collection

### Converting inputs
The below function converts strings of the form "1,234,567,890" to numerics 1234567890.

In [41]:
def twitch_numberfy(s):
    return int(s.replace(",", ""))

### Scraping
The scraping module below is responsible for collecting data about all live streams currently streaming under 1 of the categories defined above. The driver does the following steps:
* Navigate to category *C*'s browse page
    * Collect the available streams (these should be the streams with the most concurrent viewers)
    * Scroll to the bottom of the page in an attempt to load more streams
    * Repeat this process until no new streams are found
* Store category *C* streams

In [60]:
#Stream map
stream_map = {}

#Spin up browser
driver = webdriver.Chrome()

#Number of times to scroll down a page at a time
scrolls = 0
#Time to sleep between scrolls
sleeptime = 0.5

#Iterate over each category
for c in category_map:
    print("---{}---".format(c.title()))
    start = time()
    #Load up page
    driver.get(category_map[c])
    
    #Streams for storage
    streams = []
    
    try:
        #Wait for page to load at least 1 stream
        element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CLASS_NAME, "stream-thumbnail")))
        
        #Loop to get more streams
        prev_stream_count = -1
        while True:            
            #Assign global scrollable element for access through other scripts
            scroll_script = """
            window.s = document.getElementsByClassName("root-scrollable__wrapper")[0].parentElement.parentElement;
            """
            driver.execute_script(scroll_script)
            
            #Scroll
            scroll_script = """
            window.s.scrollTo(0, window.s.scrollHeight);
            """
            print("Scrolling for {} seconds ... ".format(scrolls * sleeptime))
            for i in range(scrolls):
                driver.execute_script(scroll_script)
                sleep(sleeptime)
                
            #Find streams
            streams = driver.find_elements_by_class_name("live-channel-card__videos")
            #Find text in parent containing concurrent viewers
            streams = map(lambda e: e.find_element_by_xpath("..").text, streams)
            
            #Display last scraped data
            print("Streams: {}".format(len(streams)))
            
            #If last stream obtained has 0 viewers, break
            if prev_stream_count == len(streams):
                break
            prev_stream_count = len(streams)
            break
    finally:
        print("Time for retrieval and unwrapping of streams: {}s\n\n".format(time() - start))
    
    #Iterate over streams
    for s in streams:
        #Add kvp mapping for streamer
        streamer = s.split(" ")[-1]
        #Foreign players with non-english characters have "(name_here)"
        if "(" in streamer:
            streamer = streamer[1:-1]
        
        #Load up page
        print("{}".ljust(40 - len(streamer), " ").format(streamer)),
        driver.get(stream_string.format(streamer))
        try:
            #Wait for page to load total viewer count
            #element = WebDriverWait(driver, 5).until(
            #        EC.text_to_be_present_in_element((By.CLASS_NAME, "tw-stat__value")))
            sleep(1)
            
            #Record stats
            #Followers
            followers = twitch_numberfy(
                driver.find_elements_by_class_name(
                    "channel-header__item-count")[1].find_element_by_tag_name("span").get_attribute("innerHTML"))
            #Current viewers and total views
            bottom_stats = driver.find_elements_by_class_name("tw-stat__value")
            if len(bottom_stats) == 0:
                #Hosting, so skip
                print("[HOSTING->SKIPPED]")
                continue
            current_viewers = twitch_numberfy(bottom_stats[0].get_attribute("innerHTML"))
            total_views = twitch_numberfy(bottom_stats[1].get_attribute("innerHTML"))
            
            #Store in dictionary
            stream_map[streamer] = {
                "followers": followers,
                "current_viewers": current_viewers,
                "total_views": total_views
            }
            print("[DONE]")
        except (RuntimeError, TypeError, NameError):
            print("ERROR: {} stats could not be located.".format(streamer))

#Close browser
driver.close()

---Irl---
Scrolling for 0.0 seconds ... 
Streams: 30
Time for retrieval and unwrapping of streams: 1.85899996758s


bratishkinoff                          [DONE]
wtcNN                                  [DONE]
aXtLOL                                 [DONE]
Amouranth                              [DONE]
Theokoles                              [DONE]
WingsOfRedemption                      [DONE]
HellYeahPlay                           [DONE]
DieHahn                                [DONE]
Kw3nji                                 [DONE]
The8BitDrummer                         [DONE]
StephenFRA                             [DONE]
ArtiFixOfficial                        [DONE]
Bajheera                               [DONE]
thisEguy                               [DONE]
dasMEHDI                               [DONE]
wayne379                               [DONE]
kiichichaosreigns                      [DONE]
CryptoWorldNews                        [DONE]
jasmynnive                             [DONE]
lerasplay 

In [ ]:
from time import localtime, strftime
strftime("%Y-%m-%d %H:%M:%S", localtime())